## Parte 1: Modelagem Conceitual 

### 1.1. Diagrama ER

In [ ]:
from IPython.display import Image, display

# URL da imagem
url = "https://i.postimg.cc/nz0mBWkS/der-tp1-BDA-drawio.png"

# Exibir a imagem
display(Image(url=url))

### 1.2. Decisões de modelagem
- Relacionameto  **CLIENTE - PEDIDO: 1:N**\
Este relacionamento é 1:N, pois um cliente pode realizar diversos pedidos, entretanto, um pedido só pode estar vinculado a um único cliente.\
Além disso, um cliente pode nao realizar nenhum pedido, mas todo pedido deve estar associado a um cliente.

- Relacionamento **PEDIDO - PRODUTO: N:M**\
Este relacionamento é N:M, pois um pedido pode conter diversos produtos e um tipo de produto pode estar em diversos pedidos diferentes.\
Além disso, todo pedido deve conter produtos, mas nem todo produto precisa estar em um pedido.

- Relacionamento **PRODUTO - CARGA: N:M**\
Foi-se considerado como carga o conjunto de itens comprados. Exemplo: comprou-se os itens a, b e c, logo a carga será o pacote com os itens do pedido.  
Este relacionamento é N:M, pois um mesmo tipo de produto pode estar em diversas cargas diferentes, e uma mesma carga pode conter vários tipos de produtos diferentes.
Além disso, toda carga deve conter produtos, mas nem todo produto precisa estar em uma carga.

- Relacionamento **CLIENTE - ENDEREÇO: N:1**\
Este relacionamento é N:1, pois um cliente , mas em um mesmo endereço pode residir diversos clientes.
Além disso, todo cliente deve residir em um endereço, mas nem todo endereço necessita ter um cliente.

- Relacionamento **CARGA - VEÍCULO: N:1**\
Este relacionamento é N:1, pois uma carga é transportada por apenas um veículo, mas um veículo pode transportar várias cargas.
Além disso, toda carga deve ser transportada por um veículo, mas nem todo veículo precisa transportar uma carga.

- Relacionamento **VEÍCULO - MOTORISTA: 1:1**:\
Este relacionamento é 1:1, pois um veículo está associado a apenas um motorista e um motorista está associado a apenas o seu único veículo.
Além disso, um veículo pode não estar associado a nenhum motorista, e podem haver motoristas não associados a nenhum veículo.

- Relacionamento **MOTORISTA - ROTAS: 1:N**:\
Este relacionamento é 1:N, pois um mesmo motorista pode realizar múltiplas rotas diferentes, mas uma rota está associada a um único motorista.
Além disso, este é um relacionamento fraco, uma vez que se motorista for removido do banco de dados, a rota não irá existir pois não haverá ninguém para realizá-la.

- Relacionamto **ENDEREÇO - CARGA: 1:N (Tem origem):**\
Este relacionamento é 1:N, pois um mesmo endereço pode ser origem de várias cargas diferentes, mas uma carga específica só possui um endereço de origem.
Além disso, toda carga necessariamente tem origem em um endereço, mas nem todo endereço precisa ser a origem de uma carga.

- Relacionamto **ENDEREÇO - CARGA: 1:N (Tem destino):**\
Este relacionamento é 1:N, pois um mesmo endereço pode ser destino de várias cargas diferentes, mas uma carga específica só possui um endereço de destino.
Além disso, toda carga necessariamente tem um endereço como destino, mas nem todo endereço precisa ser o destino de uma carga.



## Parte 2: Esquema Relacional

In [ ]:
from IPython.display import Image, display

# URL da imagem
url = "https://i.postimg.cc/9MnMWH12/relacional-tp1-BDA-drawio.png"

# Exibir a imagem
display(Image(url=url))

## Parte 3: Álgebra Relacional

### 3.1. Listar todos os clientes com cargas em rota de entrega
A ← Cliente ⋈ {Cliente.cpf_cliente = Pedido.id_cpf_cliente} Pedido\
B ← A ⋈ {Pedido.id_pedido = Pedido_Produto.id_pedido} Pedido_Produto\
C ← B ⋈ {Pedido_Produto.id_produto = Carga_Produto.id_produto} Carga_Produto\
D ← σ {status_carga = "em rota"} (C ⋈ {Carga_Produto.id_carga = Carga.id_carga} Carga)\
Resultado ← π {cpf_cliente, nome_cliente} (D)

### 3.2. Recuperar o nome dos produtos que nunca foram entregues
A ← Produto ⋈ {Produto.id_produto = Carga_Produto.id_produto} Carga_Produto\
B ← A ⋈ {Carga_Produto.id_carga = Carga.id_carga} Carga\
Entregues ← π {id_produto, nome_produto} (σ {status_carga = "entregue"}(B))\
Todos ← π {id_produto, nome_produto} (Produto)\
Resultado ← Todos - Entregues

### 3.3. Obter a descrição das cargas transportadas por um mesmo motorista
A ← Motorista ⋈ {Motorista.id_veiculo = Veiculo.id_veiculo} Veiculo\
B ← A ⋈ {Veiculo.id_veiculo = Carga.id_veiculo} Carga\
Resultado ← π {cpf_motorista, nome_motorista, id_carga, status_carga, data_entrega, id_end_origem, id_end_destino} (B)


### 3.4. Motoristas que entregam mais de uma carga no mesmo dia 

R1 ← Motorista ⋈_{Motorista.id_veiculo = Veiculo.id_veiculo} Veiculo\
R2 ← R1 ⋈_{Veiculo.id_veiculo = Carga.id_veiculo} σ_{status_carga='entregue' ∧ data_entrega ≠ NULL}(Carga)\
T  ← γ_{cpf_motorista, nome_motorista, data_entrega ; count(id_carga) → qnt}(R2)\
Resultado ← σ_{qnt > 1}(T)


### 3.5. Clientes que nunca receberam uma carga entregue no seu endereço

E ← σ_{status_carga='entregue' ∧ data_entrega ≠ NULL}(Carga)\
R ← Cliente ⋈_{Cliente.id_endereco = E.id_endereco_destino} E\
Ccom ← π_{cpf_cliente, nome_cliente}(R)\
Resultado ← π_{cpf_cliente, nome_cliente}(Cliente) − Ccom


### 3.6. Produtos transportados por mais de um tipo de veículo

R ← Produto ⋈_{Produto.id_produto = Carga_Produto.id_produto} Carga_Produto\
     ⋈_{Carga_Produto.id_carga = Carga.id_carga} Carga\
     ⋈_{Carga.id_veiculo = Veiculo.id_veiculo} Veiculo\
T ← γ_{id_produto, nome_produto ; count_distinct(tipo_veiculo) → qtd}(R)\
Resultado ← σ_{qtd > 1}(T)


### 3.7. Valor total gasto por cliente em pedidos

R ← Cliente ⋈_{Cliente.cpf_cliente = Pedido.cpf_cliente} Pedido\
Resultado ← γ_{cpf_cliente, nome_cliente ; sum(total_pedido) → valor_total}(R)


### 3.8. Veículos com maior número de cargas
R ← Veiculo ⋈_{Veiculo.id_veiculo = Carga.id_veiculo} Carga\
T ← γ_{id_veiculo, nome_veiculo ; count(id_carga) → qtd}(R)\
Resultado ← T


## Parte 4: SQL

### 4.1. Criação do Banco

In [ ]:
import sqlite3

conn = sqlite3.connect("meubanco.db")
cursor = conn.cursor()

# cria a tabela
cursor.executescript("""
DROP TABLE IF EXISTS Endereco;
DROP TABLE IF EXISTS Cliente;
DROP TABLE IF EXISTS Produto;
DROP TABLE IF EXISTS Pedido;
DROP TABLE IF EXISTS Veiculo;
DROP TABLE IF EXISTS Motorista;
DROP TABLE IF EXISTS Rotas;
DROP TABLE IF EXISTS Carga;
DROP TABLE IF EXISTS Pedido_Produto;
DROP TABLE IF EXISTS Carga_Produto;
               
CREATE TABLE Endereco (
    id_endereco   INTEGER PRIMARY KEY,
    nome_rua      TEXT,
    num_rua       INTEGER,
    complemento   TEXT,
    cep           INTEGER,
    bairro        TEXT
);

CREATE TABLE Cliente (
    cpf_cliente   INTEGER PRIMARY KEY,
    nome_cliente  TEXT,
    id_endereco   INTEGER,
    FOREIGN KEY (id_endereco) REFERENCES Endereco(id_endereco)
);
               
CREATE TABLE Produto (
    id_produto    INTEGER PRIMARY KEY,
    nome_produto  TEXT,
    preco_produto    INTEGER
);
               
CREATE TABLE Pedido (
    id_pedido     INTEGER PRIMARY KEY,
    data_pedido   TEXT,
    total_pedido  INTEGER,
    entrega_prevista TEXT,
    cpf_cliente INTEGER,
    FOREIGN KEY (cpf_cliente) REFERENCES Cliente(cpf_cliente)
);
               
CREATE TABLE Veiculo (
    id_veiculo    INTEGER PRIMARY KEY,
    nome_veiculo  TEXT,
    tipo_veiculo  TEXT
);

CREATE TABLE Motorista (
    cpf_motorista INTEGER PRIMARY KEY,
    nome_motorista TEXT,
    id_veiculo    INTEGER,
    FOREIGN KEY (id_veiculo) REFERENCES Veiculo(id_veiculo)
);

CREATE TABLE Rotas (
    cpf_motorista TEXT,
    id_rota INTEGER,
    nome_rota TEXT,
    PRIMARY KEY (cpf_motorista, id_rota),   -- chave composta
    FOREIGN KEY (cpf_motorista) 
        REFERENCES Motorista(cpf_motorista) 
        ON DELETE CASCADE
);
               
CREATE TABLE Carga (
    id_carga            INTEGER PRIMARY KEY,
    id_endereco_origem  INTEGER,
    id_endereco_destino INTEGER,
    id_veiculo          INTEGER,
    status_carga        TEXT,
    data_entrega        TEXT,
    FOREIGN KEY (id_endereco_origem)  REFERENCES Endereco(id_endereco),
    FOREIGN KEY (id_endereco_destino) REFERENCES Endereco(id_endereco),
    FOREIGN KEY (id_veiculo)          REFERENCES Veiculo(id_veiculo)
);
               
CREATE TABLE Pedido_Produto (
    id_pedido     INTEGER,
    id_produto    INTEGER,
    qtd_pedida    INTEGER,
    total_produto INTEGER,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido)  REFERENCES Pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES Produto(id_produto)
);               

CREATE TABLE Carga_Produto (
    id_carga   INTEGER,
    id_produto INTEGER,
    tam_carga  INTEGER,
    PRIMARY KEY (id_carga, id_produto),
    FOREIGN KEY (id_carga)   REFERENCES Carga(id_carga),
    FOREIGN KEY (id_produto) REFERENCES Produto(id_produto)
);
""")

conn.commit()


### 4.2. Inserção de Dados

Foi utilizado o ChatGPT para a geração dos dados.

In [ ]:
cursor.executescript("""                                    
-- ENDERECO
INSERT INTO Endereco (id_endereco, nome_rua, num_rua, complemento, cep, bairro) VALUES
(101, 'Rua A', 10, 'Ap 101', 30100000, 'Bairro A'),
(102, 'Rua B', 20, NULL, 30100010, 'Bairro B'),
(103, 'Rua C', 30, NULL, 30100020, 'Bairro C'),
(104, 'Rua D', 40, 'Sala 2', 30100030, 'Bairro D'),
(105, 'Rua E', 50, NULL, 30100040, 'Bairro E'),
(106, 'Rua F', 60, NULL, 30100050, 'Bairro F'),
(107, 'Rua G', 70, NULL, 30100060, 'Bairro G'),
(108, 'Rua H', 80, 'Loja 1', 30100070, 'Bairro H'),
(109, 'Rua I', 90, NULL, 30100080, 'Bairro I'),
(110, 'Rua J', 100, NULL, 30100090, 'Bairro J'),
(111, 'Rua K', 110, NULL, 30100100, 'Bairro K'),
(112, 'Rua L', 120, NULL, 30100110, 'Bairro L'),
(113, 'Rua M', 130, NULL, 30100120, 'Bairro M'),
(114, 'Rua N', 140, NULL, 30100130, 'Bairro N'),
(115, 'Rua O', 150, NULL, 30100140, 'Bairro O');

-- CLIENTE
INSERT INTO Cliente (cpf_cliente, nome_cliente, id_endereco) VALUES
(1001, 'Alice Souza', 101),
(1002, 'Bruno Lima', 102),
(1003, 'Carla Mendes', 103),
(1004, 'Diego Castro', 104),
(1005, 'Eva Pereira', 105),
(1006, 'Felipe Alves', 106),
(1007, 'Gina Duarte', 107),
(1008, 'Henrique Paz', 108),
(1009, 'Iara Torres', 109),
(1010, 'João Prado', 110),
(1011, 'Karen Vilela', 111),
(1012, 'Lucas Neri', 112),
(1013, 'Marta Reis', 113),
(1014, 'Nuno Meireles', 114),
(1015, 'Olga Silva', 115);

-- PRODUTO
INSERT INTO Produto (id_produto, nome_produto, preco_produto) VALUES
(1, 'Notebook 14\"', 3500),
(2, 'Mouse Óptico', 60),
(3, 'Teclado Mecânico', 420),
(4, 'Monitor 24\"', 900),
(5, 'Headset USB', 150),
(6, 'Cabo HDMI', 40),
(7, 'SSD 1TB', 520),
(8, 'Pen Drive 32GB', 35),
(9, 'Webcam HD', 230),
(10, 'Impressora', 780),
(11, 'Roteador Wi-Fi', 210),
(12, 'Hub USB', 90),
(13, 'Microfone', 250),
(14, 'Caixa de Som', 180),
(15, 'Estabilizador', 160);

-- VEICULO
INSERT INTO Veiculo (id_veiculo, nome_veiculo, tipo_veiculo) VALUES
(1, 'Van 01', 'Van'),
(2, 'Caminhão 3/4', 'Caminhao'),
(3, 'Fiorino 07', 'Furgao'),
(4, 'HR 2.5', 'Caminhonete'),
(5, 'Sprinter 415', 'Van'),
(6, 'Van 06', 'Van'),
(7, 'Caminhão 7/8', 'Caminhao'),
(8, 'Kombi 08', 'Furgao'),
(9, 'HR 2.6', 'Caminhonete'),
(10, 'Van 10', 'Van'),
(11, 'VUC 11', 'Caminhao'),
(12, 'Fiorino 12', 'Furgao'),
(13, 'Van 13', 'Van'),
(14, 'HR 2.7', 'Caminhonete'),
(15, 'Sprinter 416', 'Van'),
(16, 'Van 28', 'Van'),
(17, 'Caçamba 37', 'Caminhonete'),
(18, 'Caminhão 3/4', 'Caminhao');

-- MOTORISTA
INSERT INTO Motorista (cpf_motorista, nome_motorista, id_veiculo) VALUES
(2001, 'João Batista', 1),
(2002, 'Marcos Paulo', 2),
(2003, 'Nina Rocha', 3),
(2004, 'Otávio Reis', 4),
(2005, 'Paula Dias', 5),
(2006, 'Rafael Gomes', 6),
(2007, 'Sofia Lima', 7),
(2008, 'Tiago Campos', 8),
(2009, 'Úrsula Teles', 9),
(2010, 'Vitor Amaral', 10),
(2011, 'Wesley Novaes', 11),
(2012, 'Xavier Prado', 12),
(2013, 'Yasmin Couto', 13),
(2014, 'Zeca Moraes', 14),
(2015, 'Ana Paula', 15),
(2016, 'João Silva', 16),
(2017, 'Maria Souza', 17),
(2018, 'Carlos Oliveira', 18);

-- ROTAS - entidade fraca, depende do motorista
INSERT INTO Rotas (cpf_motorista, id_rota, nome_rota) VALUES
(2001, 1, 'Centro–A'),
(2002, 1, 'A–B'),
(2003, 1, 'B–C'),
(2004, 1, 'C–D'),
(2005, 1, 'D–E'),
(2006, 1, 'E–F'),
(2007, 1, 'F–G'),
(2008, 1, 'G–H'),
(2009, 1, 'H–I'),
(2010, 1, 'I–J'),
(2011, 1, 'J–K'),
(2012, 1, 'K–L'),
(2013, 1, 'L–M'),
(2014, 1, 'M–N'),
(2015, 1, 'N–O');

-- PEDIDO
INSERT INTO Pedido (id_pedido, data_pedido, total_pedido, entrega_prevista, cpf_cliente) VALUES
(5001, '2025-08-10', 1000, '2025-09-05', 1001),
(5002, '2025-08-10',  800, '2025-08-20', 1002),
(5003, '2025-08-11', 1200, '2025-09-06', 1003),
(5004, '2025-08-11',  900, '2025-08-22', 1004),
(5005, '2025-08-12', 1100, '2025-09-08', 1005),
(5006, '2025-08-12', 1300, '2025-09-09', 1006),
(5007, '2025-08-13',  700, '2025-08-15', 1007),
(5008, '2025-08-13',  650, '2025-09-10', 1008),
(5009, '2025-08-14',  500, '2025-09-11', 1009),
(5010, '2025-08-14',  450, '2025-09-12', 1010),
(5011, '2025-08-15',  400, '2025-09-13', 1011),
(5012, '2025-08-15',  300, '2025-09-14', 1012),
(5013, '2025-08-16',  200, '2025-09-15', 1013),
(5014, '2025-08-16',  150, '2025-09-16', 1014),
(5015, '2025-08-17',  100, '2025-09-17', 1015);

-- CARGA
INSERT INTO Carga (id_carga, id_endereco_origem, id_endereco_destino, id_veiculo, status_carga, data_entrega) VALUES
(6001, 105, 101, 1,  'em_rota',   NULL),
(6002, 101, 102, 2,  'entregue',  '2025-08-25'),
(6003, 108, 103, 3,  'pendente',  NULL),
(6004, 109, 104, 4,  'entregue',  '2025-08-24'),
(6005, 110, 105, 5,  'em_rota',   NULL),
(6006, 111, 106, 6,  'entregue',  '2025-08-29'),
(6007, 112, 107, 7,  'entregue',  '2025-08-18'),
(6008, 113, 108, 8,  'pendente',  NULL),
(6009, 114, 109, 9,  'em_rota',   NULL),
(6010, 115, 110, 10, 'entregue',  '2025-08-30'),
(6011, 101, 111, 11, 'pendente',  NULL),
(6012, 102, 112, 12, 'em_rota',   NULL),
(6013, 103, 113, 13, 'entregue',  '2025-08-26'),
(6014, 104, 114, 14, 'pendente',  NULL),
(6015, 105, 115, 15, 'em_rota',   NULL),
(6016, 115, 102, 16, 'em_rota', NULL),
(6017, 112, 107, 16, 'pendente', NULL),
(6018, 114, 112, 6, 'entregue', '2025-08-29'),
(6019, 113, 111, 6, 'entregue', '2025-08-29'),
(6020, 114, 109, 6, 'entregue', '2025-08-29'),
(6021, 107, 110, 3, 'entregue', '2025-08-25'),
(6022, 114, 112, 3, 'entregue', '2025-08-25'),
(6023, 114, 106, 9, 'em_rota', '2025-08-30'),
(6024, 110, 109, 9, 'em_rota', '2025-08-30'),
(6025, 115, 103, 2, 'entregue', '2025-08-26'),
(6026, 112, 103, 2, 'entregue', '2025-08-26');
                     

-- PEDIDO_PRODUTO – chaves compostas (id_pedido, id_produto)
INSERT INTO Pedido_Produto (id_pedido, id_produto, qtd_pedida, total_produto) VALUES
(5001, 1, 1, 3500),
(5002, 2, 5,  300),
(5003, 4, 1,  900),
(5004, 3, 1,  420),
(5005, 5, 2,  300),
(5006, 7, 2, 1040),
(5007, 6, 3,  120),
(5008, 8, 5,  175),
(5009, 9, 2,  460),
(5010, 10,1,  780),
(5011, 11,1,  210),
(5012, 12,2,  180),
(5013, 13,1,  250),
(5014, 14,1,  180),
(5015, 15,1,  160);

-- CARGA_PRODUTO – vincula cargas aos produtos
INSERT INTO Carga_Produto (id_carga, id_produto, tam_carga) VALUES
(6001, 5,  2),
(6002, 2,  5),
(6003, 4,  1),
(6004, 3,  1),
(6005, 6,  3),
(6006, 7,  2),
(6007, 1,  1),
(6008, 8,  5),
(6009, 9,  2),
(6010, 10, 1),
(6011, 11, 1),
(6012, 12, 2),
(6013, 13, 1),
(6014, 14, 1),
(6015, 15, 1),
(6016, 11, 2),
(6017, 12, 3);
""")

### 4.3. Consultas

In [ ]:
import pandas as pd

#### 4.3.1. Listar todos os clientes com cargas em rota de entrega

In [ ]:
query = """
SELECT DISTINCT
  c.cpf_cliente,
  c.nome_cliente
FROM Cliente c
JOIN Carga cg
  ON cg.id_endereco_destino = c.id_endereco
WHERE cg.status_carga = 'em_rota'
ORDER BY c.nome_cliente;
"""

df = pd.read_sql_query(query, conn)
df

#### 4.3.2. Recuperar o nome dos produtos que nunca foram entregues

In [ ]:
query = """
SELECT p.nome_produto
FROM Produto p
EXCEPT
SELECT p.nome_produto
FROM Produto p
JOIN Carga_Produto cp ON p.id_produto = cp.id_produto
JOIN Carga c          ON cp.id_carga  = c.id_carga
WHERE c.status_carga = 'entregue';
"""

df = pd.read_sql_query(query, conn)
df

#### 4.3.3.  Obter a descrição das cargas transportadas por um mesmo motorista

In [ ]:
query = """
SELECT
  m.cpf_motorista,
  m.nome_motorista,
  c.id_carga,
  c.status_carga,
  c.data_entrega,
  c.id_endereco_origem  AS id_end_origem,
  c.id_endereco_destino AS id_end_destino
FROM Motorista m
JOIN Veiculo  v ON v.id_veiculo = m.id_veiculo
JOIN Carga    c ON c.id_veiculo = v.id_veiculo
ORDER BY m.nome_motorista, c.id_carga;
"""

df = pd.read_sql_query(query, conn)
df

### 4.3.4. Motoristas que entregam mais de uma carga no mesmo dia 

In [ ]:
query = """
SELECT
  m.cpf_motorista,
  m.nome_motorista,
  c.data_entrega,
  COUNT(c.id_carga) AS qnt_cargas
FROM Motorista m
JOIN Veiculo  v ON v.id_veiculo = m.id_veiculo
JOIN Carga    c ON c.id_veiculo = v.id_veiculo
WHERE c.data_entrega IS NOT NULL
GROUP BY m.cpf_motorista, m.nome_motorista, c.data_entrega
HAVING COUNT(c.id_carga) > 1;
"""

df = pd.read_sql_query(query, conn)
df

### 4.3.5 Clientes que nunca receberam uma carga entregue no seu endereço

In [ ]:
query = """
SELECT c.cpf_cliente, c.nome_cliente
FROM Cliente c
WHERE NOT EXISTS (
  SELECT 1
  FROM Carga ca
  WHERE ca.id_endereco_destino = c.id_endereco
    AND ca.status_carga = 'entregue'
    AND ca.data_entrega IS NOT NULL
);


"""
df = pd.read_sql_query(query, conn)
df


### 4.3.6 Produtos transportados por mais de um tipo de veículo

In [ ]:
query = """
SELECT
  p.id_produto,
  p.nome_produto,
  COUNT(DISTINCT v.tipo_veiculo) AS qtd_tipos_veiculo
FROM Produto p
JOIN Carga_Produto cp ON cp.id_produto = p.id_produto
JOIN Carga c          ON c.id_carga    = cp.id_carga
JOIN Veiculo v        ON v.id_veiculo  = c.id_veiculo
GROUP BY p.id_produto, p.nome_produto
HAVING COUNT(DISTINCT v.tipo_veiculo) > 1
ORDER BY qtd_tipos_veiculo DESC, p.nome_produto;

"""
df = pd.read_sql_query(query, conn)
df

### 4.3.7 Valor total gasto por cliente em pedidos

In [ ]:
query = """
SELECT cl.cpf_cliente, cl.nome_cliente, SUM(p.total_pedido) AS total_gasto
FROM Cliente cl
JOIN Pedido p ON p.cpf_cliente = cl.cpf_cliente
GROUP BY cl.cpf_cliente, cl.nome_cliente
ORDER BY total_gasto DESC;
"""
df = pd.read_sql_query(query, conn)
df

### 4.3.8 Veículos com maior número de cargas

In [ ]:
query = """
SELECT v.id_veiculo, v.nome_veiculo, COUNT(c.id_carga) AS qtd_cargas
FROM Veiculo v
LEFT JOIN Carga c ON c.id_veiculo = v.id_veiculo
GROUP BY v.id_veiculo, v.nome_veiculo
ORDER BY qtd_cargas DESC;
"""

df = pd.read_sql_query(query, conn)
df

#### 4.4. View que mostra as cargas com entrega atrasada, com descrição da carga, origem, destino e nome do motorista

In [ ]:
view_sql = """
DROP VIEW IF EXISTS vw_cargas_atrasadas;

CREATE VIEW vw_cargas_atrasadas AS
WITH prazo_cliente_por_carga AS (
  SELECT
    c.id_carga,
    MIN(p.entrega_prevista) AS entrega_prevista
  FROM Carga c
  LEFT JOIN Cliente cli ON cli.id_endereco = c.id_endereco_destino
  LEFT JOIN Pedido  p   ON p.cpf_cliente   = cli.cpf_cliente
  GROUP BY c.id_carga
)
SELECT
  c.id_carga,
  ('Carga ' || c.id_carga || ' - ' || c.status_carga) AS descricao_carga,
  (eo.nome_rua || ', ' || COALESCE(eo.num_rua,'') || ' - ' || eo.bairro) AS origem,
  (ed.nome_rua || ', ' || COALESCE(ed.num_rua,'') || ' - ' || ed.bairro) AS destino,
  m.nome_motorista,
  pc.entrega_prevista,
  c.data_entrega
FROM Carga c
JOIN prazo_cliente_por_carga pc ON pc.id_carga = c.id_carga
JOIN Endereco eo ON eo.id_endereco = c.id_endereco_origem
JOIN Endereco ed ON ed.id_endereco = c.id_endereco_destino
JOIN Veiculo  v  ON v.id_veiculo   = c.id_veiculo
JOIN Motorista m ON m.id_veiculo   = v.id_veiculo
WHERE pc.entrega_prevista IS NOT NULL
  AND (
       (c.status_carga = 'entregue' AND c.data_entrega > pc.entrega_prevista)
       OR
       (c.status_carga <> 'entregue' AND date('now') > pc.entrega_prevista)
  );
"""

conn.executescript(view_sql)

df = pd.read_sql_query("SELECT * FROM vw_cargas_atrasadas", conn)
df

In [ ]:
conn.close()

## Parte 5 - Normalização

### 5.1. Tabela Pedido (Não normalizada)
PEDIDO (id_pedido, data_pedido, total_pedido, entrega_prevista, cpf_cliente, nome_cliente, FK.id_endereco, (id_produto, nome_produto, preco_produto, qtd_pedida, total_produto))

### Parte 5.2
- **1FN:**\
\
Nesta etapa, removeu-se os atributos repetitivos (multivalorados) da entidade PEDIDO, devidindo ela em PEDIDO e PEDIDO_PRODUTO, uma vez que, em um mesmo pedido, pode haver mais de um produto.\
\
PEDIDO (<u>id_pedido</u> data_pedido, total_pedido, entrega_prevista, cpf_cliente, nome_cliente, FK.id_endereco)\
\
PEDIDO_PRODUTO(<u>id_pedido, id_produto</u>, qtd_pedida, total_produto, nome_produto, preco_produto)\

- **2FN:**\
\
Nesta etapa, removeu-se as dependências parciais em PEDIDO_PRODUTO:\
\
id_produto --> {nome_produto, preco_produto}\
\
PEDIDO (<u>id_pedido</u>, data_pedido, total_pedido, entrega_prevista, cpf_cliente, nome_cliente, FK.id_endereco)\
\
PEDIDO_PRODUTO(<u>id_pedido, id_produto</u>, qtd_pedida, total_produto)\
\
PRODUTO(<u>id_produto</u>, nome_produto, preco_produto)

- **3FN:**\
\
Nesta etapa, cria-se a tabela cliente para evitar redundância.\
\
PEDIDO (<u>id_pedido</u>, data_pedido, total_pedido, entrega_prevista, FK.cpf_cliente)\
\
CLIENTE (<u>cpf_cliente</u>, nome_cliente, FK.id_endereco)\
\
PEDIDO_PRODUTO(<u>id_pedido, id_produto</u>, qtd_pedida, total_produto)\
\
PRODUTO(<u>id_produto</u>, nome_produto, preco_produto)


